In [1]:
%matplotlib inline
from collections import defaultdict, Counter
import glob
import os
import json

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam
import gspread
from gscripts.general import parsers
from gscripts.general import dataviz
%load_ext autoreload
%autoreload 2
reload(parsers)
reload(pybedtools)
import seaborn as sns

from oauth2client.client import SignedJwtAssertionCredentials

In [2]:
analysis_dir = "/home/gpratt/projects/fox2_iclip/analysis/v5/"
fox2 = parsers.clipseq_metrics(analysis_dir, iclip=True)
fox2["Fraction Collapsed"] = fox2['Usable Reads'] / fox2['Uniquely Mapped Reads'].astype(float)
fox2["Fraction Usable"] = fox2['Usable Reads'] / fox2['Input Reads'].astype(float)
fox2['percent_repetitive'] = fox2['repetitive_count'] / fox2['Reads Written']

new_index = []
for item in fox2.index:
    split_result = item.split("_")
    if len(split_result) < 6:
        new_index.append(("merged", item))
    else:
        new_index.append((split_result[4], item))

fox2.index = pd.MultiIndex.from_tuples(new_index, names=['exp', 'name'])
fox2 = fox2.sortlevel()

In [3]:
filtered_fox2 = fox2[[ "Input Reads",
                                #'Trimmed bases',
                            "Reads Written",
                            "repetitive_count",
                            'percent_repetitive',
                            "Uniquely Mapped Reads",
                            "Uniquely mapped reads %",
                            "Usable Reads",
                            "Fraction Collapsed",
                            "Fraction Usable",
                            #"spot",
                            "Num Peaks",
                            ]]

filtered_fox2.to_csv("/home/gpratt/Dropbox/encode_integration/for_eric/unmerged_fox2.csv")
HTML(filtered_fox2.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))


In [104]:
merged_fox2 = filtered_fox2.groupby(level="exp").sum()
merged_fox2["Fraction Collapsed"] = merged_fox2['Usable Reads'] / merged_fox2['Uniquely Mapped Reads'].astype(float)
merged_fox2["Fraction Usable"] = merged_fox2['Usable Reads'] / merged_fox2['Input Reads'].astype(float)
merged_fox2['percent_repetitive'] = merged_fox2['repetitive_count'] / merged_fox2['Reads Written']
merged_fox2 = merged_fox2.drop("Num Peaks", axis=1)
merged_fox2 = merged_fox2.drop("merged", axis=0)

In [105]:
merged_fox2.to_csv("/home/gpratt/Dropbox/encode_integration/for_eric/merged_fox2.csv")
merged_fox2

,Input Reads,Reads Written,repetitive_count,percent_repetitive,Uniquely Mapped Reads,Usable Reads,Fraction Collapsed,Fraction Usable
exp,,,,,,,,
highRNAse,373637261,244805863,359866240,1.470007,24130966,1118674,0.046358,0.002994
lowRNAse,415071169,281194086,171788727,0.610926,175431887,11623253,0.066255,0.028003


#IMP

In [106]:
analysis_dir = "/home/gpratt/projects/encode/analysis/imp/"
imp = parsers.clipseq_metrics(analysis_dir, iclip=True)
imp["Fraction Collapsed"] = imp['Usable Reads'] / imp['Uniquely Mapped Reads'].astype(float)
imp["Fraction Usable"] = imp['Usable Reads'] / imp['Input Reads'].astype(float)
imp['percent_repetitive'] = imp['repetitive_count'] / imp['Reads Written']

grouper = pd.read_table("/home/gpratt/projects/encode/scripts/encode_clipseq_imp.txt", header=None, names=['location', 'species', 'exp'])
grouper['run_name'] = grouper.location.apply(lambda x: ".".join(os.path.basename(x).split(".")[:-2]))
imp = pd.merge(grouper, imp, how="outer", left_on="run_name", right_index=True)
imp.index = pd.MultiIndex.from_tuples(zip(imp.exp, imp.run_name), names=['exp', 'name'])
imp = imp.sortlevel()

In [107]:
filtered_imp = imp[[ "Input Reads",
                                #'Trimmed bases',
                            "Reads Written",
                            "repetitive_count",
                            'percent_repetitive',
                            "Uniquely Mapped Reads",
                            "Uniquely mapped reads %",
                            "Usable Reads",
                            "Fraction Collapsed",
                            "Fraction Usable",
                            #"spot",
                            "Num Peaks",
                            ]]
filtered_imp.to_csv("/home/gpratt/Dropbox/encode_integration/for_eric/unmerged_imp.csv")
HTML(filtered_imp.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

In [108]:
merged_imp = filtered_imp.groupby(level="exp").sum()
merged_imp["Fraction Collapsed"] = merged_imp['Usable Reads'] / merged_imp['Uniquely Mapped Reads'].astype(float)
merged_imp["Fraction Usable"] = merged_imp['Usable Reads'] / merged_imp['Input Reads'].astype(float)
merged_imp['percent_repetitive'] = merged_imp['repetitive_count'] / merged_imp['Reads Written']
merged_imp = merged_imp.drop("Num Peaks", axis=1)
#merged_imp = merged_imp.drop("merged", axis=0)

In [109]:
merged_imp.to_csv("/home/gpratt/Dropbox/encode_integration/for_eric/merged_imp.csv")
merged_imp

,Input Reads,Reads Written,repetitive_count,percent_repetitive,Uniquely Mapped Reads,Usable Reads,Fraction Collapsed,Fraction Usable
exp,,,,,,,,
IMP1_A,354337123,327396873,181886149,0.555552,192471798,14029896,0.072893,0.039595
IMP1_B,201858824,194709533,132884041,0.682473,113391331,2284953,0.020151,0.011320
IMP2_A,139390955,124368806,89240107,0.717544,56697506,6944328,0.122480,0.049819
IMP2_B,169611402,153979631,97854397,0.635502,84311495,10237036,0.121419,0.060356
IMP3_A,397907299,385201875,330888167,0.858999,163223493,46681966,0.286000,0.117319
IMP3_B,321627896,310838264,295124266,0.949446,114833112,35935470,0.312936,0.111730
